<a href="https://colab.research.google.com/github/adarsh-meher/MachineHack---Predict-News-Category/blob/master/MH_Predict_News_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fasttext

     |████████████████████████████████| 61kB 8.6MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2385514 sha256=41676bc0603807190afcafa105d251e36402af037fe15559b1ef7b4892899c12
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


In [2]:
!git clone https://github.com/adarsh-meher/MachineHack---Predict-News-Category.git
import os
os.chdir('/content/MachineHack---Predict-News-Category/')

Cloning into 'MachineHack---Predict-News-Category'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [0]:
import torch
import pandas as pd
import numpy as np
import scipy as sc
import seaborn as sb
import matplotlib.pyplot as plt
import time
import nltk
import gensim as G
import spacy as S
import fasttext as ft

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
news_data = pd.ExcelFile('Data_Train.xlsx').parse('Sheet1')

In [6]:
news_data.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ‘...",1
4,"In global markets, gold prices edged up today ...",3


In [7]:
#### Counts of news category
print(news_data['SECTION'].value_counts()/news_data.shape[0])

'''
Politics: 0
Technology: 1
Entertainment: 2
Business: 3
'''

### no imbalance in categories

target_to_name_map = {0:'Politics',1:"Technology",2:"Entertainment",3:"Business"}

1    0.363398
2    0.252229
0    0.221028
3    0.163346
Name: SECTION, dtype: float64


In [0]:
#### Creating a test set assuming similar proportion of categories in target
news_train,news_test,target_train,target_test = train_test_split(news_data.drop(['SECTION'],axis = 1),news_data['SECTION'],test_size = 0.2,stratify = news_data['SECTION'])
news_train.index,target_train.index = range(news_train.shape[0]),range(news_train.shape[0])
news_test.index,target_test.index = range(news_test.shape[0]),range(news_test.shape[0])

## ***Analyzing news_train texts***

In [0]:
df = pd.concat([news_train.rename(columns = {'STORY' : 'text'}),pd.Series(target_train,name = 'target')],axis = 1)

In [10]:
df.head()

,text,target
0,The 5G enabled Mi Mix 3 will start rolling out...,1
1,He added that people believe in promises made ...,0
2,The not-so-great-things about Ant-ManCould hav...,2
3,Tthe Redmi Note 7 is powered by Snapdragon 660...,1
4,Researchers at the Dartmouth College have turn...,1


In [11]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()
ls = LancasterStemmer()
wns = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
##### Freq Dist of all words for all categories except stopwords
tokens = [ y for x in df['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha()]
tokens_fdist = FreqDist(tokens)

In [13]:
#### Freq distribution of words for each category
token_cat = {}
token_cat_fdist = {}
for i in df['target'].unique():
  print(target_to_name_map[i])
  token_cat[target_to_name_map[i]] = [ y for x in df[df['target'] == i]['text'].values for y in word_tokenize(x) if y.lower() not in stop_words and y.isalpha() ]
  token_cat_fdist[target_to_name_map[i]] = FreqDist(token_cat[target_to_name_map[i]]) 

Technology
Politics
Entertainment
Business


In [0]:
#### Dataframe to show for each category the top n words
n = 100
top_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common(n)],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common(n)],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common(n)],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common(n)]},index=range(n))

In [15]:
top_words_each_cat

,Technology,Politics,Entertainment,Business
0,also,BJP,film,said
1,said,Congress,said,year
2,India,said,also,market
3,Apple,party,like,India
4,new,Modi,actor,US
...,...,...,...,...
95,media,since,every,deal
96,well,go,feel,time
97,digital,many,got,yield
98,many,come,Kalank,economy


In [0]:
#### Extract set of words that appear both of any of the two categories
from itertools import combinations
common_words = []
for i in combinations(target_to_name_map.values(),2):
  [ common_words.append(x) for x in set(top_words_each_cat[i[0]]).intersection(set(top_words_each_cat[i[1]])) ]

common_words = set(list(common_words))

In [0]:
#### Dataframe to show for each category the bottom n words
n = 100
least_words_each_cat = pd.DataFrame({"Technology" : [i for i,v in token_cat_fdist["Technology"].most_common()[-n:]],
                                      "Politics" : [i for i,v in token_cat_fdist["Politics"].most_common()[-n:]],
                                      "Entertainment" : [i for i,v in token_cat_fdist["Entertainment"].most_common()[-n:]],
                                      "Business" : [i for i,v in token_cat_fdist["Business"].most_common()[-n:]]},index=range(n))

In [19]:
least_words_each_cat

,Technology,Politics,Entertainment,Business
0,Joyo,newbie,BawliTared,gallon
1,Amplug,collar,NaniKanchana,awry
2,reap,Peak,thumbs,channels
3,pentatonic,kathiri,upTrade,prominently
4,Silent,veyil,Comedy,components
...,...,...,...,...
95,Put,Dhangar,Moore,Nothing
96,Xeon,Fourth,Chrissy,WagonR
97,Conquerors,Republic,Metz,inertia
98,Total,floodgates,Sterling,unsustainably
